In [1]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

There's a new jupysql version available (0.11.1), you're running 0.10.10. To upgrade: pip install jupysql --upgrade
Deploy Panel apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


Loading configurations from /workspaces/fundamentals-of-data-transformation-4554074/pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaylimit,10
displaycon,False


,Count
0,224


Build a query to fetch all the parks in Utah (UT) and order the results by the park name.

In [2]:
%%sql
SELECT
    name, 
    states
FROM nps_public_data.parks p
WHERE p.states ILIKE '%UT%'
ORDER BY 1
LIMIT 5;

,name,states
0,Arches,UT
1,Bryce Canyon,UT
2,California,"CA,CO,ID,KS,MO,NE,NV,OR,UT,WY"
3,Canyonlands,UT
4,Capitol Reef,UT


Build a query to fetch all the National Parks that cross state boundaries (i.e. parks that are in more than one state). Remember to filter `parks` for *only* National Parks.

Hint: `parks.states` is a string representation of a list, i.e. `UT,CA,NC`. The `parks` table includes parks that aren't National Parks.

In [3]:
%%sql
SELECT
    name,
    states
FROM nps_public_data.parks p
WHERE p.states LIKE '%,%'
    AND p.designation ILIKE '%National Park%'

,name,states
0,Yellowstone,"ID,MT,WY"
1,Death Valley,"CA,NV"
2,Great Smoky Mountains,"NC,TN"


For all national parks, return the `states` column as a `STRUCT` type with each element as a state.

In [4]:
%%sql
SELECT
    name,
    states,
    split(states, ',') ::string[] AS states_list
FROM nps_public_data.parks p
WHERE designation ILIKE '%National Park%'
LIMIT 5

,name,states,states_list
0,Isle Royale,MI,[MI]
1,Black Canyon Of The Gunnison,CO,[CO]
2,Grand Canyon,AZ,[AZ]
3,Crater Lake,OR,[OR]
4,Yellowstone,"ID,MT,WY","[ID, MT, WY]"


Which parks are in **either** Montana (MT) or Wyoming (WY)?

In [5]:
%%sql

SELECT

    name,
    states,
    split(states, ',') ::string[] AS states_list

FROM nps_public_data.parks p
WHERE p.designation ILIKE '%National Park%'
AND list_has_any(states_list, ['MT', 'WY'])



,name,states,states_list
0,Yellowstone,"ID,MT,WY","[ID, MT, WY]"
1,Glacier,MT,[MT]
2,Grand Teton,WY,[WY]


What about _both_ Montana _and_ Wyoming?

In [6]:
%%sql

SELECT

    name,
    states,
    split(states, ',') ::string[] AS states_list

FROM nps_public_data.parks p
WHERE p.designation ILIKE '%National Park%'
AND list_has_all(states_list, ['MT', 'WY'])

,name,states,states_list
0,Yellowstone,"ID,MT,WY","[ID, MT, WY]"


Which park is in the greatest number of states? How many states>

In [7]:
%%sql

SELECT
    name,
    states,
    split(states, ',') ::string[] AS states_list,
    length(states_list) as num_states

FROM nps_public_data.parks p
WHERE 1=1
     AND  p.designation ILIKE '%National Park%'
ORDER BY num_states DESC


,name,states,states_list,num_states
0,Yellowstone,"ID,MT,WY","[ID, MT, WY]",3
1,Death Valley,"CA,NV","[CA, NV]",2
2,Great Smoky Mountains,"NC,TN","[NC, TN]",2
3,Isle Royale,MI,[MI],1
4,Black Canyon Of The Gunnison,CO,[CO],1
...,...,...,...,...
56,Voyageurs,MN,[MN],1
57,Mesa Verde,CO,[CO],1
58,Petrified Forest,AZ,[AZ],1
59,Virgin Islands,VI,[VI],1


Now, how many parks are in each "group" of state border-crossings? i.e. how many parks are in 1 state, 2 states, 3 states, etc.

Hint: we're grouping by the _number_ of states.

In [8]:
%%sql
with count_states AS(
SELECT
    name,
    states,
    split(states, ',') ::string[] AS states_list,
    length(states_list) as num_states

FROM nps_public_data.parks p
WHERE 1=1
     AND  p.designation ILIKE '%National Park%'
--GROUP BY num_states
ORDER BY num_states DESC)

SELECT
num_states,
count(*) as num_parks,
--ROUND(COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS pct_parks
FROM count_states
GROUP BY 1
ORDER BY 1 

,num_states,num_parks
0,1,58
1,2,2
2,3,1


What's the percentage share of the total? i.e. what *percentage* of parks are in 1 state, 2 states, etc

Hint: window functions might be helpful.

In [9]:
%%sql


with count_states AS(
SELECT
    name,
    states,
    split(states, ',') ::string[] AS states_list,
    length(states_list) as num_states

FROM nps_public_data.parks p
WHERE 1=1
     AND  p.designation ILIKE '%National Park%'
--GROUP BY num_states
ORDER BY num_states DESC)

SELECT
num_states,
count(*) as num_parks,
ROUND(COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS pct_parks
FROM count_states
GROUP BY 1
ORDER BY 1 

,num_states,num_parks,pct_parks
0,1,58,0.95
1,2,2,0.03
2,3,1,0.02


Write a query that returns the _largest_ campsite in each park. 

As a bonus, do this _without_ using a CTE (filter in the same query)

In [ ]:
%%sql
SELECT
    p.fullname as park_name,
    c.name as campground_name,
    c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable as num_campsites,
    RANK() OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC) as park_campsites_rank,
FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode
    AND p.designation LIKE 'National Park%'
QUALIFY park_campsites_rank = 1
ORDER BY num_campsites DESC, park_name, park_campsites_rank ASC
LIMIT 12;

Say you'll be in California (CA) this spring and have time for three National Parks visits. 

How many combinations of national parks can you visit? Can you return the combinations in a list ordered by the name of the first park?

Hint: self-joins and cross joins might be useful.

In [15]:
%%sql
-- What about 3?
WITH california_parks AS (
    SELECT
        *
    FROM nps_public_data.parks p
    WHERE states ILIKE '%CA%'
        AND p.designation ILIKE '%National Park%'
)
  SELECT
    cp.name as first_visit,
    cp2.name as second_visit,
    cp3.name as third_visit,
    ROW_NUMBER() OVER (ORDER BY cp.fullName ASC) as rn
FROM california_parks cp
CROSS JOIN california_parks cp2
CROSS JOIN california_parks cp3
WHERE 1 = 1
    AND cp.fullName != cp2.fullName
    AND cp.fullName != cp3.fullName
    AND cp2.fullName != cp3.fullName
ORDER BY rn asc

 

,first_visit,second_visit,third_visit,rn
0,Channel Islands,Lassen Volcanic,Sequoia & Kings Canyon,1
1,Channel Islands,Lassen Volcanic,Joshua Tree,2
2,Channel Islands,Sequoia & Kings Canyon,Yosemite,3
3,Channel Islands,Sequoia & Kings Canyon,Lassen Volcanic,4
4,Channel Islands,Sequoia & Kings Canyon,Joshua Tree,5
...,...,...,...,...
205,Yosemite,Lassen Volcanic,Channel Islands,206
206,Yosemite,Pinnacles,Sequoia & Kings Canyon,207
207,Yosemite,Lassen Volcanic,Sequoia & Kings Canyon,208
208,Yosemite,Lassen Volcanic,Death Valley,209


For our final exercise— find the combinations in alphabetical order, that is, the first letter of each visit occurs in the order of the alphabet, e.g. `[C]hannel Islands, [D]eath Valley, [J]oshua Tree` would satisfy that condition.

Hint: how can you compare strings for alphabetical order? (casing matters)

In [20]:
%%sql
WITH california_parks AS (
    SELECT
        *
    FROM nps_public_data.parks p
    WHERE states ILIKE '%CA%'
        AND p.designation ILIKE '%National Park%'
), o AS (
    SELECT
        cp.name as first_visit,
        cp2.name as second_visit,
        cp3.name as third_visit,
    FROM california_parks cp
    CROSS JOIN california_parks cp2
    CROSS JOIN california_parks cp3
    WHERE 1 = 1
        AND cp.fullName != cp2.fullName
        AND cp.fullName != cp3.fullName
        AND cp2.fullName != cp3.fullName
)
SELECT
    *,
    (lower(first_visit) < lower(second_visit)) 
        AND (lower(second_visit) < lower(third_visit)) as is_ascending,
    ROW_NUMBER() OVER (ORDER BY first_visit ASC) as rn
FROM o
WHERE is_ascending
    

,first_visit,second_visit,third_visit,is_ascending,rn
0,Channel Islands,Death Valley,Yosemite,True,1
1,Channel Islands,Death Valley,Sequoia & Kings Canyon,True,2
2,Channel Islands,Death Valley,Pinnacles,True,3
3,Channel Islands,Death Valley,Lassen Volcanic,True,4
4,Channel Islands,Death Valley,Joshua Tree,True,5
5,Channel Islands,Joshua Tree,Yosemite,True,6
6,Channel Islands,Sequoia & Kings Canyon,Yosemite,True,7
7,Channel Islands,Joshua Tree,Sequoia & Kings Canyon,True,8
8,Channel Islands,Pinnacles,Sequoia & Kings Canyon,True,9
9,Channel Islands,Joshua Tree,Pinnacles,True,10
